<a href="https://colab.research.google.com/github/ghoshankur102/Judge_It_Well/blob/main/Week2/ANN_250153.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementing ANN using PyTorch on the given data**

STEP 1: Preprocessing the Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
import pandas as pd
df = pd.read_csv('Churn_Modelling.csv')
df.head()

After uploading, make sure the file is in the current working directory, or specify the correct path when reading it. If you used the programmatic upload, the file will be in the current working directory by default.

In [ ]:
#OUTCOME HERE IS 1 IF THE PERSON HAS EXITED THE BANK OR 0 if NO
X = df.drop('Exited', axis=1).values
y = df['Exited'].values

Feeding 80% of the data for training, and remaining 20% for testing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the dataset
# Re-create X as a DataFrame, dropping irrelevant columns and the target
X = df.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)

# Apply one-hot encoding to specified categorical columns
# 'Geography', 'Gender', 'Age' and 'NumOfProducts' are treated as categorical to achieve 16 features
X = pd.get_dummies(X, columns=['Geography', 'Gender', 'Age', 'NumOfProducts'])

# Convert the processed DataFrame to a NumPy array for model training
X = X.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  #test_size = 0.2 (20%)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**STEP 2: Crafting ANN in PyTorch**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ExitPredictor(nn.Module):
    def __init__(self):
        super(ExitPredictor, self).__init__()
        self.fc1 = nn.Linear(85, 32)  # Changed from 16 to 85 features to match X_train
        self.fc2 = nn.Linear(32, 64) # 32 neurons in second hidden layer
        self.output = nn.Linear(64, 1) # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [ ]:
print(X.shape)

**STEP 3: Training the Module**

In [ ]:
model = ExitPredictor()
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    # Convert arrays to tensors
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train, dtype=torch.float32)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

**STEP 4: Evaluating the Model**

In [ ]:
with torch.no_grad():
    y_predicted = model(torch.tensor(X_test, dtype=torch.float32))
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(torch.tensor(y_test).unsqueeze(1)).sum() / float(y_test.shape[0])
    print(f'Accuracy: {acc:.4f}')

In [ ]:
## Comparing the test values for output and the predicted values
for i in range(10):
    print("y_test:", y_test[i].item(), "  y_pred:", y_predicted_cls[i].item())